In [10]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [11]:
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=2,
    n_redundant=8,
    weights=[0.9, 0.1],
    flip_y=0,
    random_state=42
)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    stratify=y,
    random_state=42,
)

In [13]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [14]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": "liblinear"},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "Random Forest",
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": "logloss"},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "XGBClassifier with SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

models

[('Logistic Regression',
  {'C': 1, 'solver': 'liblinear'},
  LogisticRegression(),
  (array([[ 1.18673836,  1.51144074,  0.78490373, ..., -0.61229492,
           -0.13830257, -0.24753395],
          [-1.28810271, -1.03855344, -2.07092052, ...,  0.49607021,
           -1.50376955,  0.62474155],
          [ 1.66393774,  1.55142135,  2.25024183, ..., -0.69955586,
            1.36600648, -0.68290518],
          ...,
          [ 0.43101615,  0.90013637, -0.42606094, ..., -0.32069604,
           -1.01508454,  0.11782042],
          [ 0.79839935,  1.5003473 , -0.45098948, ..., -0.54728572,
           -1.42140103,  0.11944858],
          [ 0.67367695,  1.27538516, -0.39960348, ..., -0.464427  ,
           -1.22522394,  0.10635794]], shape=(700, 10)),
   array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
          0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0,

In [15]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [16]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [17]:
from pprint import pprint
pprint(reports)

[{'0': {'f1-score': 0.9541284403669725,
        'precision': 0.9454545454545454,
        'recall': 0.9629629629629629,
        'support': 270.0},
  '1': {'f1-score': 0.5454545454545454,
        'precision': 0.6,
        'recall': 0.5,
        'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'f1-score': 0.749791492910759,
                'precision': 0.7727272727272727,
                'recall': 0.7314814814814814,
                'support': 300.0},
  'weighted avg': {'f1-score': 0.91326105087573,
                   'precision': 0.9109090909090909,
                   'recall': 0.9166666666666666,
                   'support': 300.0}},
 {'0': {'f1-score': 0.9799635701275046,
        'precision': 0.96415770609319,
        'recall': 0.9962962962962963,
        'support': 270.0},
  '1': {'f1-score': 0.7843137254901961,
        'precision': 0.9523809523809523,
        'recall': 0.6666666666666666,
        'support': 30.0},
  'accuracy': 0.9633333333333334,
  'macro avg': {

In [18]:
# Initialize MLflow
mlflow.set_experiment("AAnomaly Detection")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2026/02/07 11:41:02 INFO mlflow.tracking.fluent: Experiment with name 'AAnomaly Detection' does not exist. Creating a new experiment.
2026/02/07 11:41:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/07 11:41:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/1/runs/2ee70bcc04334de7b00be927e50f1559
🧪 View experiment at: http://localhost:5000/#/experiments/1


2026/02/07 11:41:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/07 11:41:06 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/1/runs/11bf2d7a6dd249cfb15c185436c3f67d
🧪 View experiment at: http://localhost:5000/#/experiments/1


2026/02/07 11:41:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/07 11:41:09 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/1/runs/db7f0cc95e024beab20798268e084388
🧪 View experiment at: http://localhost:5000/#/experiments/1


2026/02/07 11:41:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/07 11:41:12 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run XGBClassifier with SMOTE at: http://localhost:5000/#/experiments/1/runs/a3964e36b394480a9e09f4b3aa6bbbd9
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [21]:
model_name = "XGB-Smote"
run_id = input("Please enter RunID")
model_uri = f"runs:/{run_id}/model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please enter RunID a3964e36b394480a9e09f4b3aa6bbbd9


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2026/02/07 12:29:35 WARNING mlflow.tracking._model_registry.fluent: Run with id a3964e36b394480a9e09f4b3aa6bbbd9 has no artifacts at artifact path 'model', registering model based on models:/m-054ef956b84a47a2ba6dae1463a673b1 instead
2026/02/07 12:29:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


🏃 View run XGBClassifier with SMOTE at: http://localhost:5000/#/experiments/1/runs/a3964e36b394480a9e09f4b3aa6bbbd9
🧪 View experiment at: http://localhost:5000/#/experiments/1
